1) **Derives a new table of US Customers called USACusts.** notice here we're using the derived table recieved from a subquery in our FROM clause.

In [2]:
use Northwinds2022TSQLV7

select *
from (
    select Customerid, CustomerCompanyName
    from Sales.Customer
    where CustomerCountry = N'USA'
) as USACusts

(13 rows affected)

Total execution time: 00:00:00.018

Customerid,CustomerCompanyName
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


2) **Derives a table for each years number of customers.** Makes use of inline aliasing, take a peek at comments.

In [13]:
use Northwinds2022TSQLV7

-- select YEAR(OrderDate) as orderyear,
--     COUNT(distinct CustomerId) as numcusts
-- from Sales.[Order]
-- group by orderyear --obv we'll get a parsing error

select orderyear,
    COUNT(distinct custid) as numcusts
from (
    --years each customer has ordered
    select YEAR(OrderDate) as orderyear,
        CustomerId as custid --these lines will execute before our first statement so hence the aliasing is possible
    from Sales.[Order]
) as D
group by orderyear

(3 rows affected)

Total execution time: 00:00:00.026

orderyear,numcusts
2014,67
2015,86
2016,81


3) **External column aliasing, does same as above.**

In [16]:
use Northwinds2022TSQLV7

select orderyear, count(distinct custid) as numcusts
from (
    select year(OrderDate), CustomerId --notice we dont alias here
    from Sales.[Order] 
) as D (orderyear, custid) -- but here after we return the derived table!
group by orderyear

(3 rows affected)

Total execution time: 00:00:00.046

orderyear,numcusts
2014,67
2015,86
2016,81


4) **Yearly count of customers handled by Employee 3.** Uses arguments.

In [20]:
use Northwinds2022TSQLV7

DECLARE @empid AS INT = 3; --used as an argument for the table we derive

SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
      FROM Sales.[Order]
      WHERE EmployeeId = @empid) AS D
GROUP BY orderyear;
GO

(3 rows affected)

Total execution time: 00:00:00.017

orderyear,numcusts
2014,16
2015,46
2016,30


5) **Years where number of customers exceeds 70.** Using nested derived tables for inline aliasing, notice how the top layer is easiest to read.

In [21]:
use Northwinds2022TSQLV7

SELECT orderyear, numcusts
FROM (SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
      FROM (SELECT YEAR(OrderDate) AS orderyear, 
                CustomerId as custid
            FROM Sales.[Order]) AS D1
      GROUP BY orderyear) AS D2
WHERE numcusts > 70;

(2 rows affected)

Total execution time: 00:00:00.033

orderyear,numcusts
2015,86
2016,81


6) **Same as above, but not as readable as you can see due to top level aliasing.**

In [22]:
use Northwinds2022TSQLV7

SELECT YEAR(OrderDate) AS orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM Sales.[Order]
GROUP BY YEAR(OrderDate)
HAVING COUNT(DISTINCT CustomerId) > 70;

(2 rows affected)

Total execution time: 00:00:00.022

orderyear,numcusts
2015,86
2016,81
